In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import pytz
import numpy as np

from models.gan_models import *
import yfinance as yf
import MetaTrader5 as mt5

In [4]:
experiment = Experiment(
  api_key="vpNJF6XOWcHS6HqH9ZFEjwRcD",
  project_name="tail-price",
  workspace="artaasd95"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information
COMET INFO: Experiment is live on comet.com https://www.comet.com/artaasd95/tail-price/eb48bfc14f224b4b934b3473e5a7fed5



In [21]:
mt5.initialize()

print('loading current tf data')
utc_from = datetime(2021, 1, 1, tzinfo=pytz.timezone("Asia/Nicosia"))
utc_to = datetime.now(pytz.timezone("Asia/Nicosia"))

data = mt5.copy_rates_range('XAUUSD', mt5.TIMEFRAME_H4, utc_from, utc_to)
data = pd.DataFrame(data)
time_data = data.time
data.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)

loading current tf data


In [23]:
data.to_csv('xau_2021_H4.csv')

In [3]:
data = pd.read_csv('xau_2021_H4.csv')

In [4]:
data

,Unnamed: 0,time,open,high,low,close
0,0,1609718400,1904.48,1944.33,1900.62,1943.10
1,1,1609804800,1941.22,1952.90,1934.13,1949.87
2,2,1609891200,1948.82,1959.30,1900.65,1918.62
3,3,1609977600,1917.82,1927.54,1906.69,1913.35
4,4,1610064000,1913.52,1917.38,1828.05,1848.58
...,...,...,...,...,...,...
932,932,1723420800,2428.05,2472.17,2423.83,2472.06
933,933,1723507200,2470.68,2476.97,2458.46,2465.41
934,934,1723593600,2466.51,2479.94,2438.05,2447.66
935,935,1723680000,2446.85,2470.08,2432.15,2456.19


In [5]:
input_size = 1
hidden_size = 1024
seq_length = 50
num_layers = 10
batch_size = 32
num_epochs = 150
learning_rate = 0.3

# Initialize models
main_gen = MainGenerator(input_size, hidden_size, num_layers, 0.5, 1)
noise_gen = NoiseGenerator(input_size, 1024, 1, 0.3)
discriminator = Discriminator(input_size, hidden_size, 15, 0.3)

In [6]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [7]:
main_gen.to(device)
noise_gen.to(device)
discriminator.to(device)

Discriminator(
  (lstm): LSTM(1, 1024, num_layers=15, batch_first=True, dropout=0.3)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (sigmoid): Sigmoid()
)

In [8]:
optimizer_G = torch.optim.AdamW(list(main_gen.parameters()) + list(noise_gen.parameters()), lr=learning_rate)
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=learning_rate)
optimizer_noise = torch.optim.AdamW(noise_gen.parameters(), lr=learning_rate)

optim_g_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, 0.1)
optim_d_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, 0.1)
optim_noise_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_noise, 0.1)

In [9]:
cauchy_dist = torch.distributions.cauchy.Cauchy(loc=0, scale=0.5)

In [11]:
adversarial_loss = F.kl_div

In [12]:
train_data = data[:int(len(data)*0.7)]
train_data = train_data.reset_index(drop=True)
test_data = data[int(len(data)*0.7):]
test_data = test_data.reset_index(drop=True)

In [19]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    batch_start = 0
    total_loss_list = []
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(train_data)/batch_size)-1):
        price = torch.tensor(train_data[batch_start:batch_start+batch_size].close.values, dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_D.zero_grad()
        d_loss = adversarial_loss(discriminator(fake_data.detach()), price, reduction='batchmean')
        d_loss.backward()
        optimizer_D.step()
        
        # Train generators
        optimizer_G.zero_grad()
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data)), F.softmax(price), reduction='batchmean')
        g_loss = adversarial_loss(F.log_softmax(fake_data), F.softmax(price), reduction='batchmean')
        noise_loss = adversarial_loss(F.log_softmax(fake_noise), F.softmax(z1), reduction='batchmean')
        total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
        total_loss.backward()
        optimizer_G.step()
        optimizer_noise.step()

        total_loss_list.append(total_loss.item())
        g_loss_list.append(g_loss.item())
        noise_loss_list.append(noise_loss.item())
        d_loss_list.append(d_loss.item())

    optim_g_sched.step()
    optim_d_sched.step()
    optim_noise_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
    experiment.log_metric('Main Generator\Train', np.average(g_loss_list), step=epoch)
    experiment.log_metric('Noise Generator\Train', np.average(noise_loss_list), step=epoch)
    experiment.log_metric('Discriminator\Train', np.average(d_loss_list), step=epoch)
    experiment.log_metric('Total\Train', np.average(total_loss_list), step=epoch)


  0%|          | 0/150 [00:00<?, ?it/s]c:\Users\arta\anaconda3\envs\torch\Lib\site-packages\torch\nn\functional.py:2976: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
  0%|          | 0/150 [01:46<?, ?it/s]

Epoch [1/150]  Discriminator Loss: -1847683840.0000  Generator Loss: -5249550.0000


NameError: name 'experiment' is not defined

In [ ]:
# Test loop
batch_start = 0
for idx in range(int(len(test_data)/batch_size)-1):
    price = torch.tensor(test_data[batch_start:batch_start+batch_size].close, dtype=torch.float).reshape(batch_size,1)
    price = price.to(device)
    batch_start = batch_start + batch_size
    # Generate random noise inputs
    z1 = cauchy_dist.sample([batch_size, 1])
    z1 = z1.to(device)
    #z2 = cauchy_dist.sample([1])
    main_gen.eval()
    noise_gen.eval()
    discriminator.eval()
    # Generate fake data
    fake_main = main_gen(price)
    fake_noise = noise_gen(z1)
    fake_data = fake_main + fake_noise


    d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data)), F.softmax(price), reduction='batchmean')
    g_loss = adversarial_loss(F.log_softmax(fake_data), F.softmax(price), reduction='batchmean')
    noise_loss = adversarial_loss(F.log_softmax(fake_noise), F.softmax(z1), reduction='batchmean')
    total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
    total_loss.backward()
    optimizer_G.step()
    optimizer_noise.step()

optim_g_sched.step()
optim_d_sched.step()
optim_noise_sched.step()

print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
experiment.log_metric('Main Generator\Test', g_loss, step=epoch)
experiment.log_metric('Noise Generator\Test', noise_loss, step=epoch)
experiment.log_metric('Discriminator\Test', d_loss, step=epoch)
experiment.log_metric('Total\Test', total_loss, step=epoch)
# log_model(experiment, model=main_gen, model_name="Main Generator")
# log_model(experiment, model=noise_gen, model_name="Noise Generator")
# log_model(experiment, model=discriminator, model_name="Discriminator")


In [ ]:
experiment.end()

In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    batch_start = 0
    total_loss_list = []
    g_loss_list = []
    noise_loss_list = []
    d_loss_list = []
    for idx in range(int(len(data)/batch_size)-1):
        price = torch.tensor(data[batch_start:batch_start+batch_size].close, dtype=torch.float).reshape(batch_size,1)
        price = price.to(device)
        batch_start = batch_start + batch_size
        # Generate random noise inputs
        z1 = cauchy_dist.sample([batch_size, 1])
        z1 = z1.to(device)
        #z2 = cauchy_dist.sample([1])
        
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_D.zero_grad()
        d_loss = adversarial_loss(discriminator(fake_data.detach()), price, reduction='batchmean')
        d_loss.backward()
        optimizer_D.step()
        
        # Train generators
        optimizer_G.zero_grad()
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        d_loss = adversarial_loss(F.log_softmax(discriminator(fake_data)), F.softmax(price), reduction='batchmean')
        g_loss = adversarial_loss(F.log_softmax(fake_data), F.softmax(price), reduction='batchmean')
        noise_loss = adversarial_loss(F.log_softmax(fake_noise), F.softmax(z1), reduction='batchmean')
        total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
        total_loss.backward()
        optimizer_G.step()
        optimizer_noise.step()

        total_loss_list.append(total_loss.item())
        g_loss_list.append(g_loss.item())
        noise_loss_list.append(noise_loss.item())
        d_loss_list.append(d_loss.item())

    optim_g_sched.step()
    optim_d_sched.step()
    optim_noise_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
    experiment.log_metric('Main Generator\Whole Train', np.average(g_loss_list), step=epoch)
    experiment.log_metric('Noise Generator\Whole Train', np.average(noise_loss_list), step=epoch)
    experiment.log_metric('Discriminator\Whole Train', np.average(d_loss_list), step=epoch)
    experiment.log_metric('Total\Whole Train', np.average(total_loss_list), step=epoch)


In [ ]:
experiment.end()

In [ ]:
torch.save(main_gen, 'checkpoints/main_gen_xau.pth')
torch.save(noise_gen, 'checkpoints/noise_gen_xau.pth')
torch.save(discriminator, 'checkpoints/discriminator_xau.pth')